<header style="background-color: rgb(0, 62, 92); color: white; margin-top: 20px; padding:28px; ">
  <img src="images/Xlogo.png" alt="Transposition of a vector" title="Vector transposition" width="115" style="float: left;">
  <p style=" text-align: center; font-size: 32px;">   
   <strong> Deep Learning in Finance MAP548 </strong></p>
  <p style=" text-align: center; font-size: 25px;"><strong> Project 1 -  Deep pricing and calibration </strong></p>
  <p style=" text-align: center; font-size: 20px;"> Eduardo Abi Jaber </p>
</header>

# <font color='red'>PLEASE ENTER YOUR FULL NAMES HERE:</font>



- MANSARD Diane
- POTTIER Clément

<font color='red'>**DEADLINE: March 2 (5:00 pm)**</font>

<font color='red'>**PLease send both pdf ipynb files with name : Name1_Name2_Project1**</font>

# The two factor Bergomi model

The two factor Bergomi Model (L Bergomi, 2005) under the risk-neutral filtered probability space $(\Omega, \mathcal F,(\mathcal F_t)_{t\geq 0}, \mathbb Q )$  has the following dynamics:


\begin{equation}
  \begin{aligned}
    dS_t &= S_t\sqrt{V_t}dB_t, \quad S_0>0;\\
    X_t &= X_t^1 + X_t^2,\\
    V_t &= \xi \exp{\left(X_t-\frac{1}{2} \mathbb{V}[X_t] \right)},\\
      X_t^i &=  \eta_i\int_0^t e^{-\kappa_i(t-s)} dW_s,
  \end{aligned}
  \end{equation}

where $ B=\rho W + \sqrt{1-\rho^2} W^{\perp}$ with $(W,W^{\perp})$ a two-dimensional Brownian motion, $\rho \in [-1,1]$.

$X_t^i$ is a Ornstein–Uhlenbeck (Gaussian) process such that $X_t^i \sim \mathcal{N}(0, \eta_i^2\frac{(1-e^{-2\kappa_i t})}{2\kappa_i})$. Note that both $X^1$ and $X^2$ are driven by the same Brownian motion.

Recall $\mathbb V[(X+Y)] = \mathbb V[X] + \mathbb V[Y] + 2\mathbb {COV}[X,Y]$.

In addition, we fix $\kappa_2=2.6$, thus there are in total five calibratable model parameters: $(\xi, \kappa_1, \eta_1, \eta_2, \rho)$.

We are interested in computing the price of European style contingent claims, with payoff $g(S_T)$ for some function $g$:
$$
C_t = \mathbb{E} \left[g(S_T) \vert \mathcal F_t) \right].
$$

For European vanilla call options, with $g(x) = (x-K)^{+}$ with strike $K$; no closed form formula to compute $g(S_T)$

Suggested range of model parameters for training:

$\xi \in [0.03,0.25], \kappa_1 \in [10,60], \eta_1 \in [5,35],\eta_2 \in [1,5],\rho \in [-0.9,-0.1]$

# Your task

Inspired by the deep pricer for the one factor Bergomi model. Implement a deep pricer for the two factor Bergomi model. You have to generate your own training set using a scheme of your choice that you have to detail (use the fixed grid for strikes and maturities below).

Once your NN is trained, showcase the train and test error, and perform a calibration on the market implied volatility surface that was used in the one factor Bergomi and comment. 

Also provide/display the output prices of the NN of the set of parameters set 1, 2 and 3 below. 

You have to provide a notebook that compiles, together with the trained weights of your Neural Networks that we can load with the command: *model_iv.load_weights('2FBergomiNNWeights.h5')*. 

(!) The actual training of NN is not difficult, the difficult part is to get good data (and lots of it), so be careful about your simulation schemes (training might require a larger dataset than for 1 factor bergomi).

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from tensorflow.keras.optimizers import Adam

## Let's generate the dataset with moving parameters

In [2]:
N=1000
N_sims = 50
kappa_2 = 2.
T = 2
n_steps = 1000
dt = T / n_steps
tt = np.linspace(0, T, n_steps + 1)
strikes = np.array([0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5])
indices = {time: np.argmin(np.abs(tt - time)) for time in [0.1, 0.3, 0.6, 0.9, 1.2, 1.5, 1.8, 2.0]}
S0 = 100
r = 0.02

In [3]:
np.random.seed(42)

def gen_bm_path(n_steps,N_sims):
    w1 = np.random.normal(0, 1, (n_steps, N_sims))
    return w1

In [4]:
X_data = np.array([]).reshape(0, 5)
y_data = np.array([]).reshape(0, 88)

for i in tqdm(range(N), desc="Avancement", unit="iter"):
    xi = np.random.uniform(0.03, 0.25)
    kappa_1 = np.random.uniform(10, 60)
    eta_1 = np.random.uniform(5, 35)
    eta_2 = np.random.uniform(1, 5)
    rho = np.random.uniform(-0.9, -0.1)
    
    
    w1 = gen_bm_path(n_steps, N_sims)
    w2 = gen_bm_path(n_steps, N_sims)
    log_S = np.ones(N_sims).reshape(1,-1) * np.log(S0)
    
    log_S = np.ones(N_sims).reshape(1,-1) * np.log(S0)
    
    # sim Xt1
    exp1_Xt1 = np.exp(kappa_1*tt[1:])
    exp2_Xt1 = np.exp(2*kappa_1*tt)

    diff_exp2_Xt1 = np.diff(exp2_Xt1)
    std_vec_Xt1 = np.sqrt(diff_exp2_Xt1/(2*kappa_1))[:,np.newaxis] #to be broadcasted columnwise 
    exp1_Xt1 = exp1_Xt1[:,np.newaxis]
    Xt1 = np.concatenate((np.zeros(w1.shape[1])[np.newaxis, :],(1/exp1_Xt1)*np.cumsum(std_vec_Xt1*w1, axis = 0)))
    Xt1 = eta_1*Xt1
    
    # sim Xt2
    exp1_Xt2 = np.exp(kappa_2*tt[1:])
    exp2_Xt2 = np.exp(2*kappa_2*tt)

    diff_exp2_Xt2 = np.diff(exp2_Xt2)
    std_vec_Xt2 = np.sqrt(diff_exp2_Xt2/(2*kappa_2))[:,np.newaxis] #to be broadcasted columnwise 
    exp1_Xt2 = exp1_Xt2[:,np.newaxis]
    Xt2 = np.concatenate((np.zeros(w1.shape[1])[np.newaxis, :],(1/exp1_Xt2)*np.cumsum(std_vec_Xt2*w1, axis = 0)))
    Xt2 = eta_2*Xt2
    
    Xt = Xt1 + Xt2
    
    #calcul de Vt
    var1 = eta_1**2*(1-np.exp(-2*kappa_1*tt))/(2*kappa_1)
    var2 = eta_2**2*(1-np.exp(-2*kappa_2*tt))/(2*kappa_2)
    covar = eta_1*eta_2*(1-np.exp(-(kappa_1+kappa_2)*tt))/(kappa_1+kappa_2) 
    var = var1 + var2 + 2*covar

    Vt = xi * np.exp(Xt - 0.5 * var[:, np.newaxis])
    
    for j in range(n_steps):
        log_S_next = log_S[j] - 0.5 * Vt[j] * dt + np.sqrt(Vt[j] * dt) * (rho * w1[j] + np.sqrt(1 - rho**2) * w2[j])
        log_S = np.append(log_S, log_S_next.reshape(1,-1), axis=0)
    
    S = np.exp(log_S)
    prices = {}
    for time, index in indices.items():
        prices[time] = [np.exp(-r * (T - time)) * np.mean(np.maximum(S[index] - strike, 0)) for strike in strikes]
    
    values_list = [value for sublist in prices.values() for value in sublist]
    y = np.array(values_list)
    X = np.array([xi, kappa_1, eta_1, eta_2, rho]).reshape(1, -1)
    #if i == 0:
    #    X_data = X
    #    y_data = y.reshape(1, -1)
    #else:
    X_data = np.append(X_data, X, axis=0)
    y_data = np.append(y_data, y.reshape(1, -1), axis=0)
    pourcentage = (i + 1) / N * 100
    print(f"\rProgression : {pourcentage:.2f}%", end="", flush=True)

Avancement:   0%|          | 0/1000 [00:00<?, ?iter/s]

Progression : 100.00%

In [5]:
print(X_data.shape)
print(y_data.shape)

(1000, 5)
(1000, 88)


# Sample IV surface values

In [6]:
#based on the grid:
S0 = 100
strikes=np.array([0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5 ])*S0
maturities=np.array([0.1,0.3,0.6,0.9,1.2,1.5,1.8,2.0 ])

indices = {time: int(time / dt) for time in maturities}

### parameter set 1

$\xi = 0.06, \kappa_1 =41.6, \eta_1 = 18.2428, \eta_2 = 3.43, \rho = -0.7$

### parameter set 2

$\xi = 0.1, \kappa_1 = 15.6, \eta_1 = 5.5857, \eta_2 = 2.2867, \rho = -0.4$

### parameter set 3

$\xi = 0.2, \kappa_1 = 54.6, \eta_1 = 31.3496, \eta_2 = 4.5733, \rho = -0.8$

In [7]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [8]:
def print_params(x):
    print(f'xi = {x[0]}', '\n', 
          f'kappa_1 = {x[1]}', '\n', 
          f'eta_1 = {x[2]}', '\n',
          f'eta_2 = {x[3]}', '\n',
          f'rho = {x[4]}', '\n')

## Preparation of the training data

In [9]:
#training_data = # importer les données d'entraînement
X = X_data
#X=training_data[:,:4]
target_y = y_data
#target_y=training_data[:,4:]
#print(training_data.shape)
print('X shape:', X.shape)
print('Y shape:', target_y.shape)

X shape: (1000, 5)
Y shape: (1000, 88)


Split the Data between training (85%) and testing (15%) and normalise inputs between $[-1,1]$ and center/standardise outputs. 

In [10]:
#splitting the data to training and testing set
X_indexed = np.concatenate((np.arange(X.shape[0])[np.newaxis].T,X),axis = 1)
X_train, X_test, y_train, y_test = train_test_split(
    X_indexed, target_y, test_size=0.15, random_state=42)

train_index = X_train[:,0].astype(int)
test_index = X_test[:,0].astype(int)
X_train = X_train[:,1:]
X_test = X_test[:,1:]

In [11]:
#Normalising input data x to the interval of [-1, 1]
def input_norm_transform(x, x_min, x_max, norm_min, norm_max):
    return (x-x_min)/(x_max-x_min)*(norm_max-norm_min)+norm_min
def input_norm_transform_inv(x_norm, x_min, x_max, norm_min, norm_max):
    return (x_norm-norm_min)/(norm_max-norm_min)*(x_max-x_min)+x_min

x_max = np.array([np.max(X_train[:,0]),  np.max(X_train[:,1]),  np.max(X_train[:,2]), np.max(X_train[:,3]), np.max(X_train[:,4])])
x_min = np.array([np.min(X_train[:,0]),  np.min(X_train[:,1]),  np.min(X_train[:,2]), np.min(X_train[:,3]), np.min(X_train[:,4])])
x_norm_min = -1
x_norm_max = 1

x_train_transform = input_norm_transform(X_train, x_min, x_max, x_norm_min, x_norm_max)
x_test_transform = input_norm_transform(X_test, x_min, x_max, x_norm_min, x_norm_max)

In [12]:
#Let's check the range for each parameter:
print('x_max:', x_max)
print('x_min:', x_min)

x_max: [ 0.24997966 59.98343366 34.94671874  4.99627174 -0.10019679]
x_min: [ 0.03073441 10.07276595  5.09257463  1.00006501 -0.89909073]


In [26]:
#Standardising output data y to the interval, so it is centred with variance 1
def output_norm_transform(y, mean, std):
    return (y-mean)/std
def output_norm_transform_inv(y_trans, mean, std):
    return y_trans*std+mean

y_mean = np.average(y_train,axis=0)
y_std = np.std(y_train,axis=0)

y_train_transform = output_norm_transform(y_train, y_mean, y_std)
y_test_transform = output_norm_transform(y_test, y_mean, y_std)

## Building of the Neural Network

In [18]:
model_iv = keras.models.Sequential([
    keras.layers.Input(shape=(5,)),
    keras.layers.Dense(60, activation="elu"),
    keras.layers.Dense(60, activation="elu"),
    keras.layers.Dense(60, activation="elu"),
    keras.layers.Dense(60, activation="elu"),
    keras.layers.Dense(60, activation="elu"),
    keras.layers.Dense(y_train_transform.shape[1], activation="linear")
])

model_iv.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 60)             │           360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 60)             │         3,660 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 60)             │         3,660 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 60)             │         3,660 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 60)             │         3,660 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 88)             │         5,368 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,368 (79.56 KB)

 Trainable params: 20,368 (79.56 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
def error_function(y_true, y_pred):
    return tf.sqrt(tf.math.reduce_mean((y_pred - y_true) ** 2))

# Define the optimizer with a specific learning rate
optimizer = Adam(learning_rate=0.001)

model_iv.compile(optimizer="adam", loss=error_function)
train_hist = model_iv.fit(x_train_transform, y_train_transform, batch_size=16, epochs=20)

Epoch 1/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.9486
Epoch 2/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0113
Epoch 3/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9323
Epoch 4/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9726
Epoch 5/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9675
Epoch 6/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9655
Epoch 7/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9855
Epoch 8/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9618
Epoch 9/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9586
Epoch 10/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9971
Epoch 11/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0122
Epoch 12/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0198
Epoch 13/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9982
Epoch 14/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9790
Epoch 15/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0248
Epoch 16/20
54/54 ━

In [16]:
test_loss = model_iv.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 97.8410  
Test Loss: 97.72058868408203


In [17]:
model_iv.load_weights('2FBergomiNNWeights_test.h5')

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '2FBergomiNNWeights_test.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
NNParameters=[]
for i in range(0,len(model_iv.layers)):
    NNParameters.append(model_iv.layers[i].get_weights())

len(NNParameters[0][0])

### Transfer the learned NN to Numpy

In [ ]:
NNParameters=[]
for i in range(0,len(model_iv.layers)):
    NNParameters.append(model_iv.layers[i].get_weights())

NumLayers=3
def elu(x):
    #Careful function ovewrites x
    ind=(x<0)
    x[ind]=np.exp(x[ind])-1
    return x
def eluPrime(y):
    # we make a deep copy of input x
    x=np.copy(y)
    ind=(x<0)
    x[ind]=np.exp(x[ind])
    x[~ind]=1
    return x
def NeuralNetwork(x):
    input1=x
    for i in range(NumLayers):
        input1=np.dot(input1,NNParameters[i][0])+NNParameters[i][1]
        #Elu activation
        input1=elu(input1)
    #The output layer is linnear
    i+=1
    return np.dot(input1,NNParameters[i][0])+NNParameters[i][1]
def NeuralNetworkGradient(x):
    input1=x
    #Identity Matrix represents Jacobian with respect to initial parameters
    grad=np.eye(4)
    #Propagate the gradient via chain rule
    for i in range(NumLayers):
        input1=(np.dot(input1,NNParameters[i][0])+NNParameters[i][1])
        grad=(np.einsum('ij,jk->ik', grad, NNParameters[i][0]))
        #Elu activation
        grad*=eluPrime(input1)
        input1=elu(input1)
    #input1.append(np.dot(input1[i],NNParameters[i+1][0])+NNParameters[i+1][1])
    grad=np.einsum('ij,jk->ik',grad,NNParameters[i+1][0])
    #grad stores all intermediate Jacobians, however only the last one is used here as output
    return grad

# References

1)  Lorenzo Bergomi. Smile dynamics II. Risk Magazine, 2005

2) Blanka Horvath, Aitor Muguruza, and Mehdi Tomas. Deep learning volatility: a deep neural 
network perspective on pricing and calibration in (rough) volatility models. Quantitativ 
Finance, 21(1):11–27, 20218

3) Ryan McCrickerd and Mikko S Pakkanen. Turbocharging monte carlo pricing for the rough 
bergomi model. Quantitative Finance, 18(11):1877–1886, 2018